In [ ]:
import pandas as pd 

Calculate some features and concatenate the seeds. REMEMBER TO IMPUTE NULL SEEDS

This notebook is the version of the create-feature-mens made by flaty rewritten in python 

In [1]:
year = 2025
base_path = f"/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/DataPreparation/CompData/march-machine-learning-mania-{year}/"
result_regualar_men = pd.read_csv(base_path + "MRegularSeasonCompactResults.csv")
M_seed = pd.read_csv(base_path + "MNCAATourneySeeds.csv")
M_teams = pd.read_csv(base_path + "MTeams.csv")
elo_rating = pd.read_csv("/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/Feature_Eng/mine_2025_EloRating_mens_10.csv")



NameError: name 'pd' is not defined

In [ ]:
elo_rating.columns

Index(['TeamID', 'Rating', 'Season', 'TeamName', 'FirstD1Season',
       'LastD1Season'],
      dtype='object')

In [ ]:


def create_features(team_id, result_regular, seed):
    # Filter games where the team played
    tmp = result_regular[(result_regular['WTeamID'] == team_id) | (result_regular['LTeamID'] == team_id)]
    
    # Process seed data
    seed_tmp = seed[seed['TeamID'] == team_id].copy()
    seed_tmp['Seed'] = seed_tmp['Seed'].str[1:3].astype(int)
    seed_tmp = seed_tmp.drop(columns=['TeamID'])
    
    # Reorganize data for wins
    tmp2 = tmp.copy()
    tmp2 = tmp2.rename(columns={
        'WTeamID': 'TeamID', 
        'WScore': 'Score', 
        'LTeamID': 'Opp_TeamID', 
        'LScore': 'Opp_Score'
    })
    
    # Reorganize data for losses
    tmp3 = tmp.copy()
    tmp3 = tmp3.rename(columns={
        'LTeamID': 'TeamID', 
        'LScore': 'Score', 
        'WTeamID': 'Opp_TeamID', 
        'WScore': 'Opp_Score'
    })
    
    # Combine and filter
    tmp4 = pd.concat([tmp2, tmp3])
    tmp4 = tmp4[tmp4['TeamID'] == team_id].copy()
    tmp4['diff_score'] = tmp4['Score'] - tmp4['Opp_Score']
    tmp4['win'] = (tmp4['diff_score'] > 0).astype(int)
    tmp4 = tmp4.sort_values(['Season', 'DayNum'])
    
    # Calculate season metrics
    tmp5 = tmp4.groupby('Season').agg(
        count=('TeamID', 'size'),
        win_count=('win', 'sum'),
        gap_avg=('diff_score', 'mean')
    ).reset_index()
    
    tmp5['win_rate'] = tmp5['win_count'] / tmp5['count']
    
    # Join with seed data
    tmp5 = pd.merge(tmp5, seed_tmp, on='Season', how='left')
    tmp5['TeamID'] = team_id
    
    return tmp5

# Process all teams
results = []
for team_id in M_teams['TeamID']:
    tmp = create_features(team_id, result_regualar_men, M_seed)
    results.append(tmp)

res = pd.concat(results)
df_feat = pd.merge(res, elo_rating, on=['TeamID', 'Season'], how='left')
df_feat = df_feat.sort_values(['Season', 'Seed'], ascending= [False, True])




In [ ]:
tmp = df_feat[df_feat['Seed'].notnull() & (df_feat['Season'] == 2025)]

df_feat.shape, tmp.shape

((13388, 11), (68, 11))

In [ ]:
OUTPUT_PATH = "/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/Train_Set/MEN/"
df_feat.to_csv(OUTPUT_PATH + 'flaty_elo_M.csv')